# Demonstration of Givens rotation decomposition

In [1]:
import numpy
from openfermion.utils import givens_decomposition
from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

A Slater determinant can be represented by an $m \times n$ matrix with orthonormal rows. Let's obtain such a matrix $Q$ using Scipy's QR decomposition algorithm.

In [2]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:m, :]
print(Q)

[[-0.542-0.124j -0.072+0.168j  0.035-0.044j -0.403+0.069j -0.062-0.475j  0.456-0.219j]
 [ 0.099+0.269j -0.015-0.168j -0.572+0.356j -0.291+0.306j  0.343+0.208j  0.308+0.038j]
 [-0.059+0.023j  0.034+0.47j  -0.359-0.342j  0.467-0.046j  0.467-0.069j  0.035-0.29j ]]


There exist unitary matrices $V$ and $U$ such that $VQU^\dagger$ is an $m \times n$ diagonal matrix. The matrix $VQ$ represents the same Slater determinant as $Q$ up to an overall phase, but it has zeros in the upper diagonal, which saves some work in the preparation of the Slater determinant. The unitary $U$ can be written in the form

$$U = G_{N_G} \cdots G_2 G_1$$

where the $G_k$ are complex Givens rotations. We can obtain $V$, the $G_k$, and the diagonal entries of $VQU^\dagger$ using the `givens_decomposition` function.

In [3]:
# Get Givens decomposition of U
V, givens_rotations, diagonal = givens_decomposition(Q)
print(V)
print()
print(diagonal)

[[ 0.208+0.136j -0.610+0.492j  0.525+0.219j]
 [ 0.570-0.166j -0.387+0.128j -0.641-0.267j]
 [ 0.765+0.j     0.394-0.254j  0.238+0.372j]]

[-0.833-0.553j -0.981-0.194j -0.996-0.092j]


We didn't print the Givens rotations in the above block because it wouldn't look pretty. The Givens rotations are returned as a list of tuples of tuples. We will now iterate through the tuples of the list, and print the innermost tuple within each tuple as a string.

In [4]:
for parallel_set in givens_rotations:
    print(tuple(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi)
                 for i, j, theta, phi in parallel_set]))

('2, 3, 0.428, -2.664',)
('1, 2, 1.150, 0.013', '3, 4, 0.891, 0.646')
('0, 1, 1.170, 1.068', '2, 3, 1.382, -2.695', '4, 5, 0.896, -2.096')
('1, 2, 1.163, -0.720', '3, 4, 1.172, -0.704')
('2, 3, 1.167, 2.260',)


There are 5 tuples printed, and within each tuple, there are strings of the form 'i, j, theta, phi'. Each such string represents an innermost tuple, and it is a description of a complex Givens rotation of the coordinates $i$ and $j$ by angles $\theta$ and $\varphi$. The $2 \times 2$ matrix corresponding to this rotation is

$$
\begin{pmatrix}
\cos \theta & -e^{i \varphi} \sin \theta \\
\sin \theta & e^{i \varphi} \cos \theta
\end{pmatrix}.
$$

The fact that there are 5 tuples means that the circuit depth to prepare the Slater determinant corresponding to $Q$ (up to a phase) has depth 5. All of the rotations within the tuple can be performed in parallel; this is possible because the indices to be rotated are disjoint. For instance, in the third step, we can perform three rotations simultaneously, on coordinates $(0, 1)$, $(2, 3)$, and $(4, 5)$.

Let's check that $VQ$ has zeros in the upper right corner.

In [5]:
# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print(W)

[[-0.325-0.216j -0.031+0.375j  0.074-0.761j  0.189-0.293j  0.000+0.j     0.000-0.j   ]
 [-0.358-0.071j  0.117-0.139j  0.327+0.073j -0.457-0.144j -0.591-0.377j -0.000+0.j   ]
 [-0.331-0.031j -0.270+0.191j -0.066+0.036j -0.218+0.41j   0.277-0.212j  0.596-0.286j]]


Now let's check the Givens decomposition. For each set of Givens rotations that can be performed in parallel, we construct the matrices corresponding to the Givens rotations and multiply them together. Then, we multiply $W = VQ$ repeatedly on the right by these matrices and check that the correct elements are zeroed out.

In [6]:
# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n, dtype=complex)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

Number of rotations to perform in parallel: 1
[[-0.325-0.216j -0.031+0.375j  0.081-0.837j -0.000+0.j     0.000+0.j     0.000-0.j   ]
 [-0.358-0.071j  0.117-0.139j  0.102+0.101j  0.565-0.044j -0.591-0.377j -0.000+0.j   ]
 [-0.331-0.031j -0.270+0.191j -0.062+0.226j -0.023-0.408j  0.277-0.212j  0.596-0.286j]]

Number of rotations to perform in parallel: 2
[[-0.325-0.216j -0.076+0.918j -0.000-0.j    -0.000+0.j     0.000+0.j     0.000-0.j   ]
 [-0.358-0.071j -0.047-0.147j  0.149-0.086j  0.899-0.07j   0.000-0.j    -0.000+0.j   ]
 [-0.331-0.031j -0.056-0.129j -0.271+0.266j -0.087+0.005j  0.041-0.528j  0.596-0.286j]]

Number of rotations to perform in parallel: 3
[[-0.833-0.553j -0.000-0.j    -0.000+0.j    -0.000-0.j     0.000+0.j     0.000+0.j   ]
 [ 0.000+0.j    -0.389-0.077j  0.795-0.459j  0.000+0.j     0.000-0.j     0.000-0.j   ]
 [-0.000-0.j    -0.359-0.033j -0.126+0.092j -0.252+0.254j  0.066-0.845j  0.000+0.j   ]]

Number of rotations to perform in parallel: 2
[[-0.833-0.553j  0.000-0.j 

Finally, let's check that the final matrix, $VQU^\dagger$, is indeed diagonal, and that its entries match the ones returned by the function.

In [7]:
# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("V * Q * U^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

V * Q * U^\dagger matches the returned diagonal:
True
